In [32]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk import sent_tokenize,word_tokenize,pos_tag,WordNetLemmatizer
import sklearn

lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
import unicodedata
import heapq


import pickle
from nltk.corpus import stopwords
from sklearn.datasets import load_files

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [34]:
# importing tweets

tweets=pd.read_csv('Tweetsdata_csv.csv')

In [35]:
tweets.shape
tweets.columns
tweets=tweets.drop(['Important Words Phrases ','Assigned To','User Name','Tweet Counts'],axis=1)
tweets=tweets.rename(columns={'Human Understanding Sentiment':'Sentiment','Product_Line of Service':'Product'})
tweets.head()

,Tweet,Sentiment_Algo_1,Sentiment_Algo_2,Sentiment,Query_Complain,Product
0,I have created and account on kotak Mahindra. And my card no.is generated..\n\nBut it's virtual. I want to know that if I have to get a real debit card then how can I get that. And how much do I have to pay for that??,neutral,neutral,Neutral,Query,Debit card
1,"@KotakBankLtd Hi, I have ING Travel card later and it now come under @KotakBankLtd . Now my travel card has been expired and I am still outside India. Now I tried many resource Customer, grievance dept for past 3 month, but no proper solution offered. Kin",negative,negative,Negative,Complain,Travel card
2,I want open a new saving acc,positive,neutral,Neutral,Query,Saving
3,Always I find there is an authentication error.Please try to fix it.,positive,neutral,Negative,Complain,Net Banking
4,Can't change my mobile no. Through this app...,neutral,neutral,Negative,Complain,Mobile Banking


In [46]:
# Converting into String format

tweets=tweets.rename(columns={'Tweet':'tweet'})

for i in range(tweets.shape[0]):
    tweets['tweet'][i]=str(tweets['tweet'][i]).strip().lower()
    tweets['Sentiment'][i]=str(tweets['Sentiment'][i]).strip().lower()
    tweets['Sentiment_Algo_1'][i]=str(tweets['Sentiment_Algo_1'][i]).strip().lower()
    tweets['Sentiment_Algo_2'][i]=str(tweets['Sentiment_Algo_2'][i]).strip().lower()
    


In [47]:
tweets.head()

,tweet,Sentiment_Algo_1,Sentiment_Algo_2,Sentiment,Query_Complain,Product
0,i have created and account on kotak mahindra. and my card no.is generated..\n\nbut it's virtual. i want to know that if i have to get a real debit card then how can i get that. and how much do i have to pay for that??,neutral,neutral,neutral,Query,Debit card
1,"@kotakbankltd hi, i have ing travel card later and it now come under @kotakbankltd . now my travel card has been expired and i am still outside india. now i tried many resource customer, grievance dept for past 3 month, but no proper solution offered. kin",negative,negative,negative,Complain,Travel card
2,i want open a new saving acc,positive,neutral,neutral,Query,Saving
3,always i find there is an authentication error.please try to fix it.,positive,neutral,negative,Complain,Net Banking
4,can't change my mobile no. through this app...,neutral,neutral,negative,Complain,Mobile Banking


In [48]:
# tweets=tweets[:500]
tweets.shape



for i in range(tweets.shape[0]):
    if tweets['Sentiment'][i]=='negative':
        tweets['Sentiment'][i]=1
    else:
        tweets['Sentiment'][i]=0
        
# for i in range(tweets.shape[0]):
#     if tweets['Sentiment_Algo_1'][i]=='negative':
#         tweets['Sentiment_Algo_1'][i]=1
#     else:
#         tweets['Sentiment_Algo_1'][i]=0
        
# for i in range(tweets.shape[0]):
#     if tweets['Sentiment_Algo_2'][i]=='negative':
#         tweets['Sentiment_Algo_2'][i]=1
#     else:
#         tweets['Sentiment_Algo_2'][i]=0
        

tweets.head()

,tweet,Sentiment_Algo_1,Sentiment_Algo_2,Sentiment,Query_Complain,Product
0,i have created and account on kotak mahindra. and my card no.is generated..\n\nbut it's virtual. i want to know that if i have to get a real debit card then how can i get that. and how much do i have to pay for that??,neutral,neutral,0,Query,Debit card
1,"@kotakbankltd hi, i have ing travel card later and it now come under @kotakbankltd . now my travel card has been expired and i am still outside india. now i tried many resource customer, grievance dept for past 3 month, but no proper solution offered. kin",negative,negative,1,Complain,Travel card
2,i want open a new saving acc,positive,neutral,0,Query,Saving
3,always i find there is an authentication error.please try to fix it.,positive,neutral,1,Complain,Net Banking
4,can't change my mobile no. through this app...,neutral,neutral,1,Complain,Mobile Banking


In [49]:
tweets.apply(lambda x: pd.lib.infer_dtype(x.values))



tweet               string 
Sentiment_Algo_1    string 
Sentiment_Algo_2    string 
Sentiment           integer
Query_Complain      mixed  
Product             mixed  
dtype: object

REMOVING HANDLES

In [50]:
# Removing user handles
for i in range(tweets.shape[0]):
#         tweets['tweet'][i] = re.sub(r'@(\w+)','',tweets['tweet'][i])
tweets.head()

,tweet,Sentiment_Algo_1,Sentiment_Algo_2,Sentiment,Query_Complain,Product
0,i have created and account on kotak mahindra. and my card no.is generated..\n\nbut it's virtual. i want to know that if i have to get a real debit card then how can i get that. and how much do i have to pay for that??,neutral,neutral,0,Query,Debit card
1,"hi, i have ing travel card later and it now come under . now my travel card has been expired and i am still outside india. now i tried many resource customer, grievance dept for past 3 month, but no proper solution offered. kin",negative,negative,1,Complain,Travel card
2,i want open a new saving acc,positive,neutral,0,Query,Saving
3,always i find there is an authentication error.please try to fix it.,positive,neutral,1,Complain,Net Banking
4,can't change my mobile no. through this app...,neutral,neutral,1,Complain,Mobile Banking


In [51]:
pd.set_option('display.max_colwidth', -1)
tweets.head()

,tweet,Sentiment_Algo_1,Sentiment_Algo_2,Sentiment,Query_Complain,Product
0,i have created and account on kotak mahindra. and my card no.is generated..\n\nbut it's virtual. i want to know that if i have to get a real debit card then how can i get that. and how much do i have to pay for that??,neutral,neutral,0,Query,Debit card
1,"hi, i have ing travel card later and it now come under . now my travel card has been expired and i am still outside india. now i tried many resource customer, grievance dept for past 3 month, but no proper solution offered. kin",negative,negative,1,Complain,Travel card
2,i want open a new saving acc,positive,neutral,0,Query,Saving
3,always i find there is an authentication error.please try to fix it.,positive,neutral,1,Complain,Net Banking
4,can't change my mobile no. through this app...,neutral,neutral,1,Complain,Mobile Banking


 LOWER CASE & Remove puctuation

In [ ]:
print("u"+"'"+tweets['tweet'][0]+"'")

In [52]:
# Preprocessing Step 1 : LOWER CASE & Remove puctuation 

def preprocess_1(tweets):
    
    for i in range(tweets.shape[0]):
        tweets['tweet'][i] = re.sub(r'@(\w+)','',tweets['tweet'][i]) #Removing twitter handles
        tweets['tweet'][i] = tweets['tweet'][i].lower() #Lower case
        tweets['tweet'][i] = re.sub(r'[^\w\s]','',tweets['tweet'][i]) # Removing Punctuation # \w -Alphanumeric \s -whitespace
        # hyphen
        tweets['tweet'][i] = re.sub(r'\s+',' ',tweets['tweet'][i]) #Replace multiple whitespaces with single whitespace
        
        tweets['tweet'][i] = re.sub(r"that's","that is",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"there's","there is",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"what's","what is",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"where's","where is",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"it's","it is",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"who's","who is",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"i'm","i am",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"she's","she is",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"he's","he is",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"they're","they are",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"who're","who are",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"ain't","am not",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"wouldn't","would not",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"shouldn't","should not",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"can't","can not",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"couldn't","could not",tweets['tweet'][i])
        tweets['tweet'][i] = re.sub(r"won't","will not",tweets['tweet'][i])
      

preprocess_1(tweets)

# print(tweets)

In [53]:
tweets.head()

,tweet,Sentiment_Algo_1,Sentiment_Algo_2,Sentiment,Query_Complain,Product
0,i have created and account on kotak mahindra and my card nois generated but its virtual i want to know that if i have to get a real debit card then how can i get that and how much do i have to pay for that,neutral,neutral,0,Query,Debit card
1,hi i have ing travel card later and it now come under now my travel card has been expired and i am still outside india now i tried many resource customer grievance dept for past 3 month but no proper solution offered kin,negative,negative,1,Complain,Travel card
2,i want open a new saving acc,positive,neutral,0,Query,Saving
3,always i find there is an authentication errorplease try to fix it,positive,neutral,1,Complain,Net Banking
4,cant change my mobile no through this app,neutral,neutral,1,Complain,Mobile Banking


REMOVING NON ASCII characters

In [54]:

# Removing Non ASCII characters

def preprocess_2(tweets)
    for i in range(tweets.shape[0]):
        words=[]
        for word in word_tokenize(tweets['tweet'][i]):
            word=re.sub(r'[^A-Aa-z0-9]','',word)
            words.append(word)
    #         print(words)
        tweets['tweet'][i]=' '.join(words)
    

preprocess_2(tweets)


In [55]:
tweets.head()

,tweet,Sentiment_Algo_1,Sentiment_Algo_2,Sentiment,Query_Complain,Product
0,i have created and account on kotak mahindra and my card nois generated but its virtual i want to know that if i have to get a real debit card then how can i get that and how much do i have to pay for that,neutral,neutral,0,Query,Debit card
1,hi i have ing travel card later and it now come under now my travel card has been expired and i am still outside india now i tried many resource customer grievance dept for past 3 month but no proper solution offered kin,negative,negative,1,Complain,Travel card
2,i want open a new saving acc,positive,neutral,0,Query,Saving
3,always i find there is an authentication errorplease try to fix it,positive,neutral,1,Complain,Net Banking
4,cant change my mobile no through this app,neutral,neutral,1,Complain,Mobile Banking


In [ ]:
# # a=['sagar bhavsar']
# t=pos_tag(word_tokenize('sagar bhavsar'))
# t[1][1][0].lower()

LEMMATIZATION & POS TAGGING

In [56]:
# Lemmatization
def lemma(tweets)

    for i in range(tweets.shape[0]):
        x=pos_tag(word_tokenize(tweets['tweet'][i]))
        len(x)
        a=[]
    #     b=[]
        for j in range(len(x)):
            wntag=x[j][1][0].lower() # wntag contains first letter of POS
            wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
            if not wntag:
                a.append(x[j][0])
    #                  lemma = word
            else:
    #                 print(wntag)
    #                 lemma = lemmatizer.lemmatize(word, wntag)
                a.append(lemmatizer.lemmatize(x[j][0],wntag))
    #     tag.append(a)
    #     print(a)
        tweets['tweet'][i]=' '.join(a)
    #     print(b)
    # print(x[1][1])
    # len(tag)
    # tag
    # len(x)



# x=pos_tag(word_tokenize("I am not playing football"))
# x[2][1]

lemma(tweets)

In [67]:
tweets

,tweet,Sentiment_Algo_1,Sentiment_Algo_2,Sentiment,Query_Complain,Product
0,i have create and account on kotak mahindra and my card nois generate but its virtual i want to know that if i have to get a real debit card then how can i get that and how much do i have to pay for that,0,0,0,Query,Debit card
1,hi i have ing travel card later and it now come under now my travel card have be expire and i be still outside india now i try many resource customer grievance dept for past 3 month but no proper solution offer kin,1,1,1,Complain,Travel card
2,i want open a new saving acc,0,0,0,Query,Saving
3,always i find there be an authentication errorplease try to fix it,0,0,1,Complain,Net Banking
4,cant change my mobile no through this app,0,0,1,Complain,Mobile Banking
5,,0,0,0,Complain,NaN
6,how i will know about my ifsc code,0,0,0,Query,Net Banking
7,app be quick and much user friendly till last week but freeze down in between these day,0,0,0,Complain,Mobile Banking
8,increase your interest present,0,0,0,Complain,Net Banking
9,my closing balance be 201 r but reflect in available balance be only 1 rs what happen about 200 please clarify,0,0,0,Complain,CASA


END OF LEMMATIZATION & POS TAGGING

In [ ]:
# #tryout code
# # Extracting handles
# re.sub(r"@(\w+)","","@sagar I am sagar")

REMOVING STOP WORDS

In [57]:
def stop_rem(tweets)

    for i in range(tweets.shape[0]):
        wostop=[]
    #     print(tweets['tweet'][i])
        for word in word_tokenize(tweets['tweet'][i]):
            if word not in stopwords.words('English'):
                wostop.append(word)
    #     print(wostop)
        tweets['tweet'][i]=' '.join(wostop)
stop_rem(tweets)        

In [58]:
tweets.shape
x=0
for i in range(tweets.shape[0]):
    if tweets['Sentiment'][i]==1:
        x+=1
print(x/tweets.shape[0])

0.37916880451513596


In [ ]:
# tweets_copy=tweets.copy()
# tweets_copy.head()

# for i in range(tweets_copy.shape[0]):
#     tweets_copy['tweet'][i]=re.sub(r'\W',' ',tweets_copy['tweet'][i])
#     unicodedata.normalize('NFD', tweets['tweet'][i]).encode('ascii', 'ignore')
    
    
# tweets_copy.head()

In [59]:
from sklearn.model_selection import train_test_split
#Train Test Split
X_train,X_test,y_train,y_test=train_test_split(tweets['tweet'],tweets['Sentiment'])
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [85]:
X_train.shape

(1461,)

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(min_df=3,max_df=0.8,ngram_range=(1,4),sublinear_tf=True,use_idf=True)


In [76]:
train_vectors=vectorizer.fit_transform(X_train)
test_vectors=vectorizer.transform(X_test)
# len(vect.get_feature_names())

In [96]:
# X_train_vectorized

<1461x3759 sparse matrix of type '<class 'numpy.float64'>'
	with 13451 stored elements in Compressed Sparse Row format>

In [77]:
# X_train_vectorized=vect.transform(X_train)

model = LogisticRegression()
model.fit(train_vectors,y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [68]:
# print(X_train_vectorized)

In [78]:
# model.score(test_vectors,y_test)

In [79]:
# Confusion Matrix
predictions=model.predict(test_vectors)
confusion_matrix=pd.crosstab(y_test,predictions,rownames=['ACTUAL'],colnames=['PREDICTED'],margins=True)

In [84]:
# predictions.tolist()

In [80]:
sklearn.metrics.accuracy_score(y_test, predictions, normalize=True, sample_weight=None)

0.78893442622950816

In [81]:
print(confusion_matrix)

PREDICTED    0    1  All
ACTUAL                  
0          264  50   314
1          53   121  174
All        317  171  488


In [ ]:
# Applying Naive Bayes

In [72]:
from sklearn.naive_bayes import MultinomialNB

model2=MultinomialNB(alpha=2)
model2.fit(train_vectors,y_train)

MultinomialNB(alpha=2, class_prior=None, fit_prior=True)

In [73]:
predictions2=model2.predict(test_vectors)
confusion_matrix=pd.crosstab(y_test,predictions2,rownames=['ACTUAL'],colnames=['PREDICTED'],margins=True)

In [74]:
sklearn.metrics.accuracy_score(y_test, predictions2, normalize=True, sample_weight=None)

0.79303278688524592